## Import the required libraries

In [26]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

## Maximise column widths

In [27]:
pd.set_option('max_colwidth', 800)

## Table preparation

In [28]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])


32

## Extracting data from Wikipedia

In [29]:
table = soup.find('table', class_ = 'wikitable sortable') 
rows = table.find_all('tr')

postcodes = [] 
boroughs = [] 
neighbourhoods = [] 

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  
            
            postcode = columns[0].text
            postcodes.append(postcode)
            
            borough = columns[1].text
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] 
            
            if neighbourhood == 'Not assigned': 
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : 
        pass 
    
postcode_explored = [] 
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] 
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) 
        postcode_explored.append(postcode_i)

## Close file

In [30]:
csv_file.close()

## Creating file and getting its shape

In [31]:
df = pd.read_csv('toronto_postal_codes.csv')
df.shape

(180, 3)

In [32]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A\n,Not assigned\n,NaN
1,M2A\n,Not assigned\n,NaN
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
